In [1]:
import pandas as pd
import requests
import bs4
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup as soup
from pytrends.request import TrendReq
import numpy as np
import datetime
import time
import csv
import dateutil.relativedelta
import matplotlib.pyplot as plt; plt.rcdefaults()
plt.rcParams['font.family'] = 'helvetica'
from sklearn.linear_model import LinearRegression
from scipy import stats
import statsmodels.formula.api as sm
import re

from pathlib import Path

In [2]:
#static lists
#for knowyourmeme
uNames = []
Names = []
#for lifetime
emptyList = []
tooNoisy = []
newList = []

# **Scraping for Meme Names**

In [3]:
def uneditedNames(memes):
    for meme in memes:
        cats = meme.find_all("td")
        for cat in cats:
            if(cat.a != None):
                items = cat.a.attrs.keys()
                if "href" in items:
                    uNames.append(cat.a['href'])
                
def scrapeMemes(numRequests):
    
    headers = {'me':'Chrome/5.0'}  
    numRequests = numRequests #288
    
    for x in range(1,numRequests):
        print(x)
        time.sleep(1)
        page = ('https://knowyourmeme.com/memes/page/' + str(x))
        req = Request(page,headers=headers)
        webpage = urlopen(req).read()
        page_soup = soup(webpage, 'html.parser')
        memes = page_soup.body.find(id="content").find(id="maru").find(id="infinite-scroll-wrapper").find(id="entries").find(id="entries_list").tbody.find_all("tr")
        uneditedNames(memes)
        
    for name in uNames:
        if ("/memes/" in name):
            newName = name
            newName = newName.replace("/memes/","")
            if re.search(r'/?/',newName):
                ""
            else:
                newName = newName.replace("-"," ")
                #newName = newName + " meme"
                Names.append(newName)

# Helper Functions

In [4]:
def totalNames(Names): #create ; DANGER: Will delete all files when ran
    with open('TotalNames', 'w') as myfile:
        wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
        wr.writerow(Names)

In [5]:
def checkNoise(x):
    try:
        return ((x.iloc[1].values[0] + x.iloc[2].values[0]) / 2) < 50
    except IndexError:
        return False
    

In [6]:
def finalDf(toIter, mainDf):
    for index, row in toIter.iterrows():
        temp = mainDf[index:]
        if checkNoise(temp):
            return temp
        
    return pd.DataFrame()

    

In [7]:
def halfLife(start,end):
    return end - start

In [8]:
def checkDups(tList,toCheck):
    nope = []
    yep = []
    for meme in tList:
        if meme in toCheck:
            yep.append(meme)
        else:
            nope.append(meme)
            
    print(len(nope))
    return nope

In [9]:
def calcDate(peak,time):
    time1 = peak - dateutil.relativedelta.relativedelta(months=1)
    time2 = peak + dateutil.relativedelta.relativedelta(months=time-1)
    time1 = time1.strftime("%Y-%m-%d ")
    time2 = time2.strftime("%Y-%m-%d")
    return time1 + time2

In [10]:
def to_date(x):
    return x[0].date()

In [11]:
def make_f_vals(df):
    fTime = []
    fPeak = []
    year = []
    newFP = []
    
    for index, row in df.iterrows():
        fTime.append(row['Half_Life'].total_seconds()/ datetime.timedelta(days=1).total_seconds())
        fPeak.append((row['Peak_Date']).toordinal())
        year.append(row['Peak_Date'].year)
    
    for x in fPeak:
        newFP.append(x-min(fPeak))
    
    df['fHalf_Life'] = fTime
    df['fPeak_Date'] = fPeak
    df['year'] = year
    df['Ordinal_Peak'] = newFP
    
    return df

# Finding lifetime of a meme

In [12]:
pytrends = TrendReq(hl='en-US', tz=360, retries=5, backoff_factor=0.1)

In [13]:
def runMemesL(memeList):
    counter = 0
    count = 0
    memeHalfLifes = pd.DataFrame(columns=['Peak_Date','Title','Half_Life'])
    for meme in memeList:
        try:
            kw_list = [meme]
            pytrends.build_payload(kw_list,  timeframe='all')
            memeInterest = pytrends.interest_over_time()
            count = count + 1
            counter = 0
            print(count)
            if memeInterest.empty:
                emptyList.append(meme)
            else:
                peakInterest = memeInterest[memeInterest[memeInterest.columns[0]] == 100]
                PeakInterestDf = memeInterest[peakInterest.index[0]:]
                halfInterest = PeakInterestDf[PeakInterestDf[memeInterest.columns[0]] < 50]
                finalInterest = finalDf(halfInterest, memeInterest)
                if finalInterest.empty:
                    tooNoisy.append(meme)
                else:
                    halfLifeVal = halfLife(peakInterest.iloc[0].name, finalInterest.iloc[0].name)
        
                    l = [[peakInterest.index[0]], memeInterest.columns[0], halfLifeVal]
                    temp = pd.DataFrame(data = [l], columns = ['Peak_Date','Title', 'Half_Life'])
                    memeHalfLifes = memeHalfLifes.append(temp)
        except:
            if counter > 3:
                return memeHalfLifes
            print(counter)
            print(meme)
            newList.append(meme)
            counter = counter + 1
            time.sleep(15)
            
    return memeHalfLifes

In [14]:
def runMemesD(memeFrame):
    counter = 0
    count = 0
    memeHalfLifes = pd.DataFrame(columns=['Peak_Date','Title','Half_Life'])
    for ir in memeFrame.itertuples():
        try:
            kw_list = [ir[2]]
            date = calcDate(ir[1],8)
            pytrends.build_payload(kw_list,  timeframe=date)
            memeInterest = pytrends.interest_over_time()
            count = count + 1
            counter = 0
            print(count)
            if memeInterest.empty:
                emptyList.append(ir[2])
            else:
                peakInterest = memeInterest[memeInterest[memeInterest.columns[0]] == 100]
                PeakInterestDf = memeInterest[peakInterest.index[0]:]
                halfInterest = PeakInterestDf[PeakInterestDf[memeInterest.columns[0]] < 50]
                finalInterest = finalDf(halfInterest, memeInterest)
                if finalInterest.empty:
                    tooNoisy.append(meme)
                else:
                    halfLifeVal = halfLife(peakInterest.iloc[0].name, finalInterest.iloc[0].name)
        
                    l = [[peakInterest.index[0]], memeInterest.columns[0], halfLifeVal]
                    temp = pd.DataFrame(data = [l], columns = ['Peak_Date','Title', 'Half_Life'])
                    memeHalfLifes = memeHalfLifes.append(temp)
        except Exception as e:
            print(e)
            if counter > 3:
                return memeHalfLifes
            print(counter)
            print(ir[2])
            counter = counter + 1
            time.sleep(15)
            
    return memeHalfLifes 

In [ ]:
with open('NamesLeft', 'r') as f:
    reader = csv.reader(f)
    your_list = list(reader)
    tList = your_list[0]
    #tList = ['book']
    
memesFA = runMemesL(tList)

test = []
for x in memesFA['Peak_Date']:
    test.append(x[0].date())
memesFA['Peak_Date'] = test

#memesFA = runMemesL(['cat'])
if Path("memes.pkl").is_file():
    df = pd.read_pickle('memes.pkl')
    df = df.append(memesFA)
else:
    df = memesFA


with open('NamesLeft', 'w') as myfile:
    wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
    wr.writerow(checkDups(checkDups(checkDups(tList,df['Title'].unique()),emptyList),tooNoisy))
    
if Path("emptyList").is_file():     
    with open('emptyList', 'r') as f:
        reader = csv.reader(f)
        your_list = list(reader)
        if len(your_list)>0:
            emptyListF = emptyList + your_list[0]
        
with open('emptyList', 'w') as myfile:
    wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
    wr.writerow(emptyList)

if Path("tooNoisy").is_file():     
    with open('tooNoisy', 'r') as f:
        reader = csv.reader(f)
        your_listN = list(reader)
        if len(your_listN)>0:
            tooNoisy = tooNoisy + your_listN[0]
        
with open('tooNoisy', 'w') as myfile:
    wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
    wr.writerow(tooNoisy)
    
df.drop_duplicates(subset='Title', keep="first", inplace=True)

df = df.reset_index(drop=True)
df.to_pickle('memes.pkl')
print(len(df))  

In [ ]:
temp_df = pd.read_pickle('memes.pkl')

with open('curr_position.txt','r') as f:
    curr = int(f.read())

temp_memes = runMemesD(temp_df[curr:])

curr = curr + len(temp_memes)
print(curr)
with open('curr_position.txt','w') as f:
    f.write(str(curr))
    
test = []
for x in temp_memes['Peak_Date']:
    test.append(x[0].date())
temp_memes['Peak_Date'] = test
if Path("final_memes.pkl").is_file(): 
    full_memes = pd.read_pickle('final_memes.pkl')
else:
    full_memes = []
        
full_memes = full_memes.append(temp_memes)
full_memes.drop_duplicates(subset='Title', keep="first", inplace=True)
full_memes = make_f_vals(full_memes)
full_memes = full_memes.reset_index(drop=True)
full_memes.to_pickle('final_memes.pkl')